# Bayesian NN dummy run

## Import packages

In [27]:
#!pip install torch torchvision torchaudio scikit-learn skorch pyro-ppl psutil

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from skorch import NeuralNetRegressor
from skorch.callbacks import EarlyStopping
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import psutil
import os

In [34]:
# Function to check system resource usage
def check_system_resources():
    cpu_usage = psutil.cpu_percent(interval=1)
    memory_info = psutil.virtual_memory()
    print(f"CPU Usage: {cpu_usage}%")
    print(f"Memory Usage: {memory_info.percent}%")
    print(f"Available Cores: {os.cpu_count()}")

# Load data
df = pd.read_csv(r"C:\Users\benja\Documents\MSC BAM\THESIS\Data\Dummy_Data_for_Bayesian_NN.csv")

# Preprocessing
encoder = OneHotEncoder(sparse_output=False, drop='first')
categorical_features = ['type', 'database_writes', 'datagrid']
categorical_encoded = encoder.fit_transform(df[categorical_features])

scaler = StandardScaler()
numeric_features = ['lines_of_code', 'amount_of_nodes']
numeric_scaled = scaler.fit_transform(df[numeric_features])

X = np.hstack((categorical_encoded, numeric_scaled))
y = df[['cu_usage', 'performance_time']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Bayesian Neural Network Definition
class BayesianNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # Two output values
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# Bayesian training using Pyro
def model(X, y=None):
    priors = {
        "fc1.weight": dist.Normal(0, 1).expand([64, X.shape[1]]).to_event(2),
        "fc1.bias": dist.Normal(0, 1).expand([64]).to_event(1),
        "fc2.weight": dist.Normal(0, 1).expand([32, 64]).to_event(2),
        "fc2.bias": dist.Normal(0, 1).expand([32]).to_event(1),
        "fc3.weight": dist.Normal(0, 1).expand([2, 32]).to_event(2),
        "fc3.bias": dist.Normal(0, 1).expand([2]).to_event(1),
    }
    
    lifted_module = pyro.random_module("module", BayesianNN(X.shape[1]), priors)
    sampled_model = lifted_module()
    
    with pyro.plate("data", X.shape[0]):
        prediction_mean = sampled_model(X)
        pyro.sample("obs", dist.Normal(prediction_mean, 1), obs=y)

# Guide function for variational inference
def guide(X, y=None):
    guide_means = {}
    guide_scales = {}

    # Define a BayesianNN instance
    bayesian_nn = BayesianNN(X.shape[1])

    for name, param in bayesian_nn.named_parameters():
        guide_means[name] = pyro.param(f"{name}_mean", torch.randn_like(param))
        guide_scales[name] = pyro.param(
            f"{name}_scale", torch.ones_like(param), constraint=dist.constraints.positive
        )

    # Create priors for parameters
    priors = {
        name: dist.Normal(guide_means[name], guide_scales[name]).to_event(1)
        for name in guide_means
    }

    # Return Bayesian module with learned priors
    lifted_module = pyro.random_module("module", bayesian_nn, priors)

    return lifted_module()

# Training the Bayesian Neural Network
pyro.clear_param_store()
optimizer = Adam({'lr': 0.01})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

num_epochs = 100
for epoch in range(num_epochs):
    try:
        total_loss = 0
        for X_batch, y_batch in train_loader:
            loss = svi.step(X_batch, y_batch)
            total_loss += loss
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")
    except Exception as e:
        print(f"Error during training at epoch {epoch+1}: {str(e)}")

# Evaluate model
with torch.no_grad():
    check_system_resources()
    for X_batch, y_batch in test_loader:
        y_pred = guide(X_batch)(X_batch)  # Pass input to the model
        mse = ((y_pred - y_batch) ** 2).mean().item()
        print(f"MSE on test batch: {mse:.4f}")


C:\Users\benja\anaconda3\Lib\site-packages\pyro\primitives.py:526: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  warnings.warn(


Error during training at epoch 1: Shape mismatch inside plate('data') at site obs dim -1, 32 vs 2
           Trace Shapes:           
            Param Sites:           
           Sample Sites:           
module$$$fc1.weight dist    | 64  6
                   value    | 64  6
  module$$$fc1.bias dist    | 64   
                   value    | 64   
module$$$fc2.weight dist    | 32 64
                   value    | 32 64
  module$$$fc2.bias dist    | 32   
                   value    | 32   
module$$$fc3.weight dist    |  2 32
                   value    |  2 32
  module$$$fc3.bias dist    |  2   
                   value    |  2   
               data dist    |      
                   value 32 |      
Error during training at epoch 2: Shape mismatch inside plate('data') at site obs dim -1, 32 vs 2
           Trace Shapes:           
            Param Sites:           
           Sample Sites:           
module$$$fc1.weight dist    | 64  6
                   value    | 64  6
  module$$$f